In [ ]:
import math
import logging
import numpy as np
import pandas as pd
import scipy.signal as sig
import altair as alt
import flammkuchen as fl

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.load import graph_melt
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.graph import graph_spinorama, graph_freq
from spinorama.compute_scores import scores
from spinorama.filter_scores import scores_apply_filter, scores_print, scores_loss

In [ ]:
df = fl.load('../cache.parse_all_speakers.h5')

In [ ]:
df_speaker = df['Adam S2V']['ASR']['asr']
df_speaker.keys()
original_mean = df_speaker['CEA2034_original_mean']

In [ ]:
g_params = {'xmin': 20, 'xmax': 20000, 'ymin': -50, 'ymax': 10, 'width': 400, 'height': 250}
score = scores(df_speaker)
# print(score)
# print(score['pref_score'])

In [ ]:
from spinorama.filter_iir import Biquad

my_fs = 48000
my_peq = parse_eq_iir_rews('../datas/eq/Adam S2V/iir.txt', my_fs)
config  =  {'1_freq': 219.55929842190855, '2_freq': 517.5686338139874, '3_freq': 1758.0618073754263, '4_freq': 3543.3617805427602, '5_freq': 5846.110212465421, '6_freq': 8745.622216400916, '1_Q': 4.307275812375801, '2_Q': 9.017958637108, '3_Q': 1.4738724299352968, '4_Q': 1.8797524543356448, '5_Q': 1.7875346196376038, '6_Q': 5.376300685731374, '1_dbGain': 0.18579947496158056, '2_dbGain': 0.3758943841525575, '3_dbGain': -1.550634775091205, '4_dbGain': -2.170016932036555, '5_dbGain': -1.0002544282506474, '6_dbGain': -1.8471388268287416}
auto_peq = [
    (1.0, Biquad(Biquad.PEAK, config['1_freq'], 48000, config['1_Q'], config['1_dbGain'])),
    (1.0, Biquad(Biquad.PEAK, config['2_freq'], 48000, config['2_Q'], config['2_dbGain'])),
    (1.0, Biquad(Biquad.PEAK, config['3_freq'], 48000, config['3_Q'], config['3_dbGain'])),
    (1.0, Biquad(Biquad.PEAK, config['4_freq'], 48000, config['4_Q'], config['4_dbGain'])),
    (1.0, Biquad(Biquad.PEAK, config['5_freq'], 48000, config['5_Q'], config['5_dbGain'])),
    (1.0, Biquad(Biquad.PEAK, config['6_freq'], 48000, config['6_Q'], config['6_dbGain'])),
]
spin_filtered, pir_filtered, score_filtered = scores_apply_filter(df_speaker, my_peq)
spin_auto, pir_auto, score_auto = scores_apply_filter(df_speaker, auto_peq)
# print(score_filtered)

In [ ]:
scores_print(score, score_filtered)
scores_print(score, score_auto)

In [ ]:
graph_spinorama(df_speaker['CEA2034'], g_params) & graph_spinorama(spin_filtered, g_params) & graph_spinorama(spin_auto, g_params)

In [ ]:
from ray import tune
from spinorama.filter_iir import Biquad

objective function is where df_speaker is fixed
scores_apply_filter(df_speaker, my_peq)

In [ ]:
def optimizer(config):
    current_peq = [
        (1.0, Biquad(Biquad.PEAK, config['1_freq'], 48000, config['1_Q'], config['1_dbGain'])),
        (1.0, Biquad(Biquad.PEAK, config['2_freq'], 48000, config['2_Q'], config['2_dbGain'])),   
    ]
    intermediate_score = scores_loss(df_speaker, current_peq)
    tune.report(mean_loss=intermediate_score)


analysis = tune.run(
    optimizer,
    config={
        "1_freq": tune.grid_search([200, 399, 10]),
        "2_freq": tune.grid_search([400, 1000, 20]),
        "1_Q": tune.grid_search([0.1, 10]),
        "2_Q": tune.grid_search([0.1, 10]),
        "1_dbGain": tune.grid_search([0.1, 10]),
        "2_dbGain": tune.grid_search([0.1, 10]),
        # "n_peq": tune.choice([2])
    })

print("Best config: ", analysis.get_best_config(metric="mean_loss"))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()